In [ ]:
import getpass
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm
from openai import OpenAI

In [2]:
MODEL       = 'gpt-3.5-turbo-instruct'
LABEL       = 'product'
COLUMN      = 'conformal_40%'
CV_SPLITS   = [0]#, 1, 2, 3, 4]
SHOTS       = 2
TEMPERATURE = 0

# Load prompts: 

In [3]:
prompts = pd.read_csv(f'../results/{MODEL.lower()}/{MODEL.lower()}_{LABEL}_{SHOTS:d}-shot.csv', index_col=0)
prompts

,cv_split,label,prompt_conformal_35%,prompt_conformal_50%,prompt_max-10,prompt_max-5,prompt_sim-10,prompt_sim-20,output_conformal_35%,output_conformal_50%,output_max-10,output_max-5,output_sim-10,output_sim-20
0,0,other types of meat,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,chilli products,chilli powder,chilli products,chilli products,canned chilli products,chilli products
1,0,Not classified pork meat,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,jam,jam,jam,jam,bacon marmalade,bacon jam
2,0,prepared dish,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,beef products,beef products,beef products,beef products,beef products,beef products
3,0,ground beef meat,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,ground beef,ground beef,ground beef,ground beef,ground beef,ground beef
4,0,chicken breast,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,chicken breast,chicken breast,chicken breast,chicken breast,chicken breast,chicken breast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,0,muesli,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,cereal,Breakfast cereals,cereals,cereal,breakfast cereals,breakfast cereals
1506,0,cheese,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,cheese,cheese,cheese,cheese,cheese,cheese
1507,0,tapioca chips,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,potato chips,tapioca chips,tapioca chips,tapioca chips,tapioca chips,tapioca chips
1508,0,mung bean sprouts,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,bean sprouts,bean sprouts,sprouts,bean sprouts,bean sprouts,bean sprouts


In [4]:
prompts_col = pd.read_csv(f'prompts_{LABEL}_{SHOTS:d}-shot.csv', index_col=0)[['cv_split', 'label', f'prompt_{COLUMN}']]
prompts_col

,cv_split,label,prompt_conformal_40%
0,0,other types of meat,Context start:\nWe are looking for food produc...
1,0,Not classified pork meat,Context start:\nWe are looking for food produc...
2,0,prepared dish,Context start:\nWe are looking for food produc...
3,0,ground beef meat,Context start:\nWe are looking for food produc...
4,0,chicken breast,Context start:\nWe are looking for food produc...
...,...,...,...
7542,4,cakes,NaN
7543,4,oats,NaN
7544,4,pomegranate,NaN
7545,4,ice cream,NaN


# Run prompts through GPT:

In [5]:
row_mask = np.bitwise_or.reduce([prompts.cv_split.values == split for split in CV_SPLITS])
row_mask.sum()

1510

In [6]:
if f'prompt_{COLUMN}' in prompts.columns: print(f'Column "prompt_{COLUMN}" already in DataFrame')
else: prompts[f'prompt_{COLUMN}'] = [''] * len(prompts)

for split in CV_SPLITS:
    m_0 = prompts.cv_split.values == split
    m_1 = prompts_col.cv_split.values == split

    assert all(prompts.label.values[m_0] == prompts_col.label.values[m_1])

    prompts.loc[m_0, f'prompt_{COLUMN}'] = prompts_col.loc[m_1, f'prompt_{COLUMN}']

In [7]:
client = OpenAI(api_key=getpass.getpass('Enter the API-key:'))

In [ ]:
prompts_out = []
results_out = []
for prompt in tqdm(prompts.loc[row_mask, f'prompt_{COLUMN}'].values):
    # adapt prompt to gpt
    prompt = prompt.replace('Context start:\n', '')
    prompt = prompt.replace('Here are some labelled examples sorted from most probable to least probable:\n', 'Please predict the correct class for the following sample:\n')
    prompt = prompt.replace('Context end:\nPlease predict the correct class for the following sample:\n', '')
    prompt = prompt.replace(' -> ', ' => ')

    prompts_out.append(prompt)

    # get unique classes:
    classes = np.unique([example.split(' => ')[1] for example in prompt.split('\n')[1:-1]])

    # if more than one class -> feed through gpt:
    if len(classes) == 1: results_out.append(classes[0])
    else: results_out.append(client.completions.create(
            model=MODEL,
            prompt=prompt,
            temperature=TEMPERATURE
        ).choices[0].text.strip())


In [9]:
col_name = f'output_{COLUMN}'

if col_name not in prompts.columns:
    prompts[col_name] = [''] * len(prompts)

prompts.loc[row_mask, f'output_{COLUMN}'] = results_out
prompts.loc[row_mask, f'prompt_{COLUMN}'] = prompts_out

# Save results:

In [10]:
def sort(s:str):
    key = ''

    if   s[0:7] == 'output_': key = '2_' + s[7:]
    elif s[0:7] == 'prompt_': key = '1_' + s[7:]
    else:                     key = '0_' + s

    return key

columns = [col for col in prompts.columns if not col.endswith('_limited')]
columns.sort(key = sort)
prompts = prompts[columns]
prompts

,cv_split,label,prompt_conformal_35%,prompt_conformal_40%,prompt_conformal_50%,prompt_max-10,prompt_max-5,prompt_sim-10,prompt_sim-20,output_conformal_35%,output_conformal_40%,output_conformal_50%,output_max-10,output_max-5,output_sim-10,output_sim-20
0,0,other types of meat,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,chilli products,chilli products,chilli powder,chilli products,chilli products,canned chilli products,chilli products
1,0,Not classified pork meat,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,jam,jam,jam,jam,jam,bacon marmalade,bacon jam
2,0,prepared dish,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,beef products,beef products,beef products,beef products,beef products,beef products,beef products
3,0,ground beef meat,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,ground beef,ground beef,ground beef,ground beef,ground beef,ground beef,ground beef
4,0,chicken breast,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,chicken breast,chicken breast,chicken breast,chicken breast,chicken breast,chicken breast,chicken breast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,0,muesli,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,cereal,cereal,Breakfast cereals,cereals,cereal,breakfast cereals,breakfast cereals
1506,0,cheese,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,cheese,cheese,cheese,cheese,cheese,cheese,cheese
1507,0,tapioca chips,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,We are looking for food products in texts. Ple...,potato chips,tapioca chips,tapioca chips,tapioca chips,tapioca chips,tapioca chips,tapioca chips
1508,0,mung bean sprouts,We are looking f

In [11]:
prompts.to_csv(f'../results/{MODEL.lower()}/{MODEL.lower()}_{LABEL}_{SHOTS:d}-shot.csv')